In [58]:
import pandas as pd

# URL to the raw routes data
url = "https://raw.githubusercontent.com/jpatokal/openflights/master/data/routes.dat"

# Define column names based on the dataset structure
columns = [
    "Airline", "AirlineID", "SourceAirport", "SourceAirportID",  
    "DestinationAirport", "DestinationAirportID",  "Codeshare", 
    "Stops", "Equipment"
]

# Read the dataset into a pandas DataFrame
routes_df = pd.read_csv(url, header=None, names=columns, delimiter=',', na_values='\\N')

# Add a column to indicate if the SourceAirport is IATA or ICAO
routes_df['SourceType'] = routes_df['SourceAirport'].apply(
    lambda x: 'IATA' if len(str(x)) == 3 else 'ICAO')


# Add a column to indicate if the DestinationAirport is IATA or ICAO
routes_df['DestinationType'] = routes_df['DestinationAirport'].apply(
    lambda x: 'IATA' if len(str(x)) == 3 else 'ICAO')


# Display the first few rows to ensure the new columns are added correctly
print(routes_df.head())
routes_df.info()


  Airline  AirlineID SourceAirport  SourceAirportID DestinationAirport  \
0      2B      410.0           AER           2965.0                KZN   
1      2B      410.0           ASF           2966.0                KZN   
2      2B      410.0           ASF           2966.0                MRV   
3      2B      410.0           CEK           2968.0                KZN   
4      2B      410.0           CEK           2968.0                OVB   

   DestinationAirportID Codeshare  Stops Equipment SourceType DestinationType  
0                2990.0       NaN      0       CR2       IATA            IATA  
1                2990.0       NaN      0       CR2       IATA            IATA  
2                2962.0       NaN      0       CR2       IATA            IATA  
3                2990.0       NaN      0       CR2       IATA            IATA  
4                4078.0       NaN      0       CR2       IATA            IATA  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67663 entries, 0 to 67662

In [59]:
import pandas as pd

# URL to the raw airports data
url = "https://raw.githubusercontent.com/jpatokal/openflights/master/data/airports.dat"

# Define column names based on the dataset structure
columns = [
    "AirportID", "Name", "City", "Country", "IATA","ICAO", "Latitude", 
    "Longitude", "Altitude", "Timezone", "DST", "TzDatabaseTimeZone", 
    "Type", "Source"
]

# Read the dataset into a pandas DataFrame
airports_df = pd.read_csv(url, header=None, names=columns, delimiter=',', na_values='\\N')

# Display the first few rows to ensure it's loaded correctly
print(airports_df.head())
airports_df.info()

   AirportID                                         Name          City  \
0          1                               Goroka Airport        Goroka   
1          2                               Madang Airport        Madang   
2          3                 Mount Hagen Kagamuga Airport   Mount Hagen   
3          4                               Nadzab Airport        Nadzab   
4          5  Port Moresby Jacksons International Airport  Port Moresby   

            Country IATA  ICAO  Latitude   Longitude  Altitude  Timezone DST  \
0  Papua New Guinea  GKA  AYGA -6.081690  145.391998      5282      10.0   U   
1  Papua New Guinea  MAG  AYMD -5.207080  145.789001        20      10.0   U   
2  Papua New Guinea  HGU  AYMH -5.826790  144.296005      5388      10.0   U   
3  Papua New Guinea  LAE  AYNZ -6.569803  146.725977       239      10.0   U   
4  Papua New Guinea  POM  AYPY -9.443380  147.220001       146      10.0   U   

     TzDatabaseTimeZone     Type       Source  
0  Pacific/Port_More

In [60]:
 #Conditional merge for SourceAirport (latitude and longitude)
if 'IATA' in routes_df['SourceType'].values:
    routes_df = routes_df.merge(
        airports_df[['IATA', 'Latitude', 'Longitude']],
        how='left',
        left_on='SourceAirport',
        right_on='IATA'
    ).rename(columns={'Latitude': 'SourceLatitude', 'Longitude': 'SourceLongitude'})
else:
    routes_df = routes_df.merge(
        airports_df[['ICAO', 'Latitude', 'Longitude']],
        how='left',
        left_on='SourceAirport',
        right_on='ICAO'
    ).rename(columns={'Latitude': 'SourceLatitude', 'Longitude': 'SourceLongitude'})

# Conditional merge for DestinationAirport (latitude and longitude)
if 'IATA' in routes_df['DestinationType'].values:
    routes_df = routes_df.merge(
        airports_df[['IATA', 'Latitude', 'Longitude']],
        how='left',
        left_on='DestinationAirport',
        right_on='IATA'
    ).rename(columns={'Latitude': 'DestinationLatitude', 'Longitude': 'DestinationLongitude'})
else:
    routes_df = routes_df.merge(
        airports_df[['ICAO', 'Latitude', 'Longitude']],
        how='left',
        left_on='DestinationAirport',
        right_on='ICAO'
    ).rename(columns={'Latitude': 'DestinationLatitude', 'Longitude': 'DestinationLongitude'})

# Drop unnecessary columns (IATA, ICAO, Altitude)
routes_df = routes_df.drop(['IATA', 'ICAO', 'IATA_x','IATA_y','AirlineID','Codeshare'], axis=1, errors='ignore')

# Display the first few rows to ensure the data is merged correctly
print(routes_df.head())

# Display the info to check the DataFrame structure
routes_df.info()


  Airline SourceAirport  SourceAirportID DestinationAirport  \
0      2B           AER           2965.0                KZN   
1      2B           ASF           2966.0                KZN   
2      2B           ASF           2966.0                MRV   
3      2B           CEK           2968.0                KZN   
4      2B           CEK           2968.0                OVB   

   DestinationAirportID  Stops Equipment SourceType DestinationType  \
0                2990.0      0       CR2       IATA            IATA   
1                2990.0      0       CR2       IATA            IATA   
2                2962.0      0       CR2       IATA            IATA   
3                2990.0      0       CR2       IATA            IATA   
4                4078.0      0       CR2       IATA            IATA   

   SourceLatitude  SourceLongitude  DestinationLatitude  DestinationLongitude  
0       43.449902        39.956600            55.606201             49.278702  
1       46.283298        48.006302

In [61]:
import pandas as pd
import numpy as np

# Function to calculate the Haversine distance
def haversine(lat1, lon1, lat2, lon2):
    # Convert latitude and longitude from degrees to radians
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    
    # Haversine formula
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat / 2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    
    # Earth's radius in kilometers
    radius = 6371.0
    distance = radius * c  # Distance in kilometers
    
    return distance

# Apply the haversine function to each row in the DataFrame
routes_df['Distance'] = routes_df.apply(
    lambda row: haversine(row['SourceLatitude'], row['SourceLongitude'], row['DestinationLatitude'], row['DestinationLongitude']),
    axis=1
)

# Display the first few rows to check the calculated distances
print(routes_df[['SourceAirport', 'DestinationAirport', 'SourceLatitude', 'SourceLongitude', 'DestinationLatitude', 'DestinationLongitude', 'Distance']].head())

# Display the info to check the DataFrame structure
routes_df.info()


  SourceAirport DestinationAirport  SourceLatitude  SourceLongitude  \
0           AER                KZN       43.449902        39.956600   
1           ASF                KZN       46.283298        48.006302   
2           ASF                MRV       46.283298        48.006302   
3           CEK                KZN       55.305801        61.503300   
4           CEK                OVB       55.305801        61.503300   

   DestinationLatitude  DestinationLongitude     Distance  
0            55.606201             49.278702  1506.825604  
1            55.606201             49.278702  1040.438320  
2            44.225101             43.081902   448.164909  
3            55.606201             49.278702   770.508500  
4            55.012600             82.650703  1338.631467  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67663 entries, 0 to 67662
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0

In [62]:
# Remove rows where the 'Distance' column is NaN
routes_df = routes_df.dropna(subset=['Distance'])

# Display the first few rows to ensure the rows have been removed
print(routes_df.head())

# Display the info to confirm the change
routes_df.info()


  Airline SourceAirport  SourceAirportID DestinationAirport  \
0      2B           AER           2965.0                KZN   
1      2B           ASF           2966.0                KZN   
2      2B           ASF           2966.0                MRV   
3      2B           CEK           2968.0                KZN   
4      2B           CEK           2968.0                OVB   

   DestinationAirportID  Stops Equipment SourceType DestinationType  \
0                2990.0      0       CR2       IATA            IATA   
1                2990.0      0       CR2       IATA            IATA   
2                2962.0      0       CR2       IATA            IATA   
3                2990.0      0       CR2       IATA            IATA   
4                4078.0      0       CR2       IATA            IATA   

   SourceLatitude  SourceLongitude  DestinationLatitude  DestinationLongitude  \
0       43.449902        39.956600            55.606201             49.278702   
1       46.283298        48.0063

In [63]:

average_speed = 850  

# Calculate flight time in hours
routes_df['FlightTime'] = routes_df['Distance'] / average_speed

# Convert flight time to hours and minutes
def format_flight_time(hours):
    """Converts flight time in decimal hours to a formatted string in 'Xh Ym'."""
    integer_hours = int(hours)
    minutes = round((hours - integer_hours) * 60)  # Round to avoid floating-point precision issues
    return f"{integer_hours}h {minutes}m"

# Apply the conversion to each row
routes_df['FlightTime'] = routes_df['FlightTime'].apply(format_flight_time)

# Display the first few rows with flight time
print(routes_df[['SourceAirport', 'DestinationAirport', 'Distance', 'FlightTime']].head())



  SourceAirport DestinationAirport     Distance FlightTime
0           AER                KZN  1506.825604     1h 46m
1           ASF                KZN  1040.438320     1h 13m
2           ASF                MRV   448.164909     0h 32m
3           CEK                KZN   770.508500     0h 54m
4           CEK                OVB  1338.631467     1h 34m


In [64]:
# Remove duplicate rows where SourceAirport and DestinationAirport are the same
routes_df = routes_df.drop_duplicates(subset=['SourceAirport', 'DestinationAirport'], keep='first')

# Display the filtered dataframe
print(routes_df)


      Airline SourceAirport  SourceAirportID DestinationAirport  \
0          2B           AER           2965.0                KZN   
1          2B           ASF           2966.0                KZN   
2          2B           ASF           2966.0                MRV   
3          2B           CEK           2968.0                KZN   
4          2B           CEK           2968.0                OVB   
...       ...           ...              ...                ...   
67653      ZL           TSV           3330.0                WIN   
67654      ZL           WGA           3363.0                MEL   
67656      ZL           WIN           6337.0                LRE   
67657      ZL           WIN           6337.0                TSV   
67658      ZL           WYA           6334.0                ADL   

       DestinationAirportID  Stops Equipment SourceType DestinationType  \
0                    2990.0      0       CR2       IATA            IATA   
1                    2990.0      0       CR2 

In [65]:
import os

# Define the file path as just the file name (no directory)
file_path = 'routes and distances.csv'  # This saves the file in the same folder as the notebook

# Now, save the DataFrame as a CSV file in the current directory
routes_df.to_csv(file_path, index=False)